## Traffic Counting Demo

This demo shows the ability of YOLOE-11 to count objects such as cars on a freeway.

For the video, I use a 20 second video of a freeway where a object detection region is drawn over both the inbound and outbound lanes.  Without modification to YOLOE-11, cars going into (towards the viewer of the video) and out of the zone are counted.  If you want to try other videos, [Pexels](https://www.pexels.com/) is a great source for royalty free files.

In this step, we load the nightly builds of torch and torchvision to support a Nvidia Blackwell card.

In [ ]:
pip install --quiet -U --pre torch torchvision --index-url https://download.pytorch.org/whl/nightly/cu128 

Now, install opencv and ultralytics.  Ultralytics will provide the YOLOE-11 model as well as the ability to count objects via ``ObectCounter`` 

In [ ]:
pip install --quiet -U opencv-python ultralytics

Now we import some other libraries that are already present in my python environment.  We set the ``YOLO_VERBOSE`` environment variable to stop the output of model inforamtion from YOLOE.

In [ ]:
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np

from ultralytics import solutions
from ultralytics import YOLOE
from copy import deepcopy

os.environ['YOLO_VERBOSE'] = 'False'


This is the main function that opens the video file, sets up the region where counting will take place and writes the video back out as an .avi file.  Once the vide ends, or a blank frame is encountered, the program will break and end.

Some of the arguments can be changed:

``region_points`` change to your liking.  I found that the region works well for the video that I used.

``conf`` - confidence of the model.  Adjust to optimize for correct object detections.

``iou`` - intersection over union for detecting overlapping objects

``show_conf`` and ``show_labels`` - shows the confidence score and label on the bounding box for each obejct that is detected.

``verbose`` - set to false to supress the output of detections while the notebook is running.  Lack of objects in the frame is still logged.

In [33]:
def count_objects_in_region(video_path, output_video_path):
    
    # Count objects in a specific region within a video.
    cap = cv2.VideoCapture(video_path)
    assert cap.isOpened(), "Error reading video file"
    w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))
    video_writer = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

    # define the region where counting will take place.
    region_points = [(20, 490), (1880, 490), (1880, 470), (20, 470)]
    
    # Counter solution from Ultralytics
    counter = solutions.ObjectCounter(
        region=region_points,
        conf=.10,
        iou = .90,
        show_conf=False,
        show_labels=False,
        verbose=False,
        model=detectionmodel
    )

    while cap.isOpened():
        success, im0 = cap.read()
        if not success:
            print("Video frame is empty or processing is complete.")
            break
        results = counter(im0)
        video_writer.write(results.plot_im)

    cap.release()
    video_writer.release()
    cv2.destroyAllWindows()

The following block of code is where we set the model and the object classes that we want to detect.  By specifying ``car`` the program will not detect trucks, including tractor-trailers or utility trucks.  Pickup trucks seem to be considered cars by YOLOE.

``verbose=False`` is specified to try to supress the output of model information when the model starts.  This does not seem to work though.

Try changing the object classes in the ``objects`` array.  

The input (``videos/traffic-20sec.mp4``) and output (``output/traffic_count_output.avi``) filenames are passed to our function.  Experiment here with different video files to see how YOLOE performs. 

In [ ]:
# Load a pretrained YOLOE model that provides image segmentation.
detectionmodel = YOLOE('yoloe-11s-seg.pt', verbose=False)

# Define the obects that we want to detect and add them ot the model.
objects = ["car"]
detectionmodel.set_classes(objects, detectionmodel.get_text_pe(objects))

# call the function to count 
count_objects_in_region("videos/traffic-20sec.mp4", "output/traffic_count_output.avi")